# Document Intelligence 画像の分析

Azure AI Document Intelligence の最新のレイアウトモデル（prebuilt-layout）は Microsoft の強力な光学式文字認識 (OCR) 機能の強化バージョンと、ディープラーニングモデルを組み合わせ、テキスト、テーブル、チェックマーク、ドキュメント構造を抽出します。今回は最新の Markdown 機能や図、セクションなどの構造解析データを分析してみましょう。

レイアウトモデルによって検出される図形オブジェクトには、boundingRegions (ページ番号や図形の境界を囲む多角形座標など、ドキュメント ページ上の図形の空間位置)、spans (ドキュメントのテキスト内でのオフセットと長さを指定する、図形に関連するテキスト スパンの詳細) などの主要なプロパティがあります。

https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-layout?view=doc-intel-4.0.0#figures

In [ ]:
# !pip install azure.ai.documentintelligence pymupdf Pillow --upgrade

In [ ]:
import azure.ai.documentintelligence
print("Azure Document Intelligence version: ", azure.ai.documentintelligence.__version__)

In [ ]:
# import libraries
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest, AnalyzeResult

# set `<your-endpoint>` and `<your-key>` variables with the values from the Azure portal
endpoint = "<your-endpoint>"
key = "<your-key>"

document_intelligence_client = DocumentIntelligenceClient(endpoint=endpoint, credential=AzureKeyCredential(key))

## URL から呼ぶ方法

In [ ]:
docUrl = "https://documentintelligence.ai.azure.com/documents/samples/layout/layout-pageobject.pdf"

poller = document_intelligence_client.begin_analyze_document(
    "prebuilt-layout", AnalyzeDocumentRequest(url_source=docUrl),output_content_format="markdown"
)
result: AnalyzeResult = poller.result()

## ファイルを直接アップロードする方法

In [ ]:
path_to_sample_documents = "./layout-pageobject.pdf"
#prebuilt-layout
with open(path_to_sample_documents, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", analyze_request=f, content_type="application/octet-stream"
    )
result: AnalyzeResult = poller.result()

### Debug用コード

[jsonpickle](https://pypi.org/project/jsonpickle/) ライブラリを使用すると `AnalyzeResult` オブジェクト構造をそのまま JSON へ保存できます。Document Intelligence の分析データを一旦自分の手元に置いて細かく調べたい時に有用です。VSCode の [Json Editor](https://marketplace.visualstudio.com/items?itemName=nickdemayo.vscode-json-editor) 拡張が便利です。

In [ ]:
import jsonpickle
# Debug 用(AnalyzeResultオブジェクト構造をそのままJSONへ)
json_data = jsonpickle.encode(result)
with open('analyzed_data_pageobject.json', "w", encoding='utf-8') as f:
    f.write(json_data)

# JSON からオブジェクト構造を復元
# f = open("analyzed_data_pageobject.json")
# json_str = f.read()
# result = jsonpickle.decode(json_str)

## テキスト行の手書きスタイル
応答では、各テキスト行が手書きスタイルであるかどうかと、信頼度スコアが分類されます。


In [ ]:
for idx, style in enumerate(result.styles):
    print(
        "Document contains {} content".format(
            "handwritten" if style.is_handwritten else "no handwritten"
        )
    )

## ページ
ページ コレクションは、サービス応答に表示される最初のオブジェクトです。レイアウトモデルでは、印刷および手書きのスタイルテキストが `lines` および `words` として抽出されます。このモデルでは、抽出された単語の境界 `polygon` 座標と `confidence` を出力します。

### 選択マーク
ドキュメントから選択マークも抽出されます。抽出された選択マークは、各ページの `pages` コレクション内に示されます。これには、境界 `polygon`、`confidence`、および選択 `state` (`selected/unselected`) が含まれます。関連するテキスト (抽出された場合) も、開始インデックス (`offset`) と `length` として含まれます。`length` はドキュメントのテキスト全体を含む最上位の `content` プロパティを参照します。

In [ ]:
for page in result.pages:
    print(f"----Analyzing layout from page #{page.page_number}----")
    print(f"Page has width: {page.width} and height: {page.height}, measured with unit: {page.unit}")

    if page.lines:
        for line_idx, line in enumerate(page.lines):
            print(f"...Line # {line_idx} {line.content}")

    if page.selection_marks:
        for selection_mark in page.selection_marks:
            print(
                f"Selection mark is '{selection_mark.state}' within bounding polygon "
                f"'{selection_mark.polygon}' and has a confidence of {selection_mark.confidence}"
            )

## テーブル
レイアウトモデルでは、JSON 出力の `pageResults` セクションにテーブルが抽出されます。抽出されるテーブル情報には、列と行の数、行の範囲、列の範囲が含まれます。境界ポリゴンのある各セルは、その領域が `columnHeader` として認識されているかどうかにかかわらず、情報と共に出力されます。このモデルでは、回転されるテーブルの抽出がサポートされています。各テーブル セルには、行と列のインデックスと境界ポリゴン座標が含まれています。セル テキストの場合、このモデルは開始インデックス (`offset`) を含む `span` 情報を出力します。

In [ ]:
if result.tables:
    for table_idx, table in enumerate(result.tables):
        print(f"Table # {table_idx} has {table.row_count} rows and {table.column_count} columns")
        if table.bounding_regions:
            for region in table.bounding_regions:
                print(f"Table # {table_idx} location on page: {region.page_number} is {region.polygon}")
        for cell in table.cells:
            print(f"...Cell[{cell.row_index}][{cell.column_index}] has text '{cell.content}'")
            if cell.bounding_regions:
                for region in cell.bounding_regions:
                    print(
                        f"...content on page {region.page_number} is within bounding polygon '{region.polygon}'\n"
                    )
print("----------------------------------------")

### 別の表記

In [ ]:
def get_tables(result):
    tables = []
    for table_idx, table in enumerate(result.tables):
        cells = []
        for cell in table.cells: 
            cells.append( {
                "row_index": cell.row_index,
                "column_index": cell.column_index,
                "content": cell.content,
            })
        tab = {
                "row_count": table.row_count,
                "column_count": table.column_count,
                "cells": cells
        }
        tables.append(tab)
        return tables
    
get_tables(result)

## 段落
`AnalyzeResult` の最上位オブジェクトとして、段落ごとのテキストブロックを抽出します。このコレクション内の各エントリはテキスト ブロックを表し、抽出されたテキスト (`content`) と `polygon` 矩形座標を含みます。`role` にはタイトル、セクション見出し、ページ ヘッダー、ページ フッターなどの属性を表す論理ロールが格納されます。

In [ ]:
def get_paragraphs(result):
    paragraphs = []
    for idx, paragraph in enumerate(result.paragraphs):
        item = {
            "id": "/paragraphs/" + str(idx),
            "content": paragraph.content if paragraph.content else "",
            "role": paragraph.role if paragraph.role else "",
            "polygon": paragraph.get("boundingRegions")[0]["polygon"],
            "pageNumber": paragraph.get("boundingRegions")[0]["pageNumber"]
        }
        paragraphs.append(item)
    return paragraphs

get_paragraphs(result)

## セクション
階層型ドキュメント構造分析の実行結果が格納されます。非構造化ドキュメントの階層構造を理解するために便利なデータ構造が得られます。

In [ ]:
def get_sections(result):
    sections = []
    for section in result.sections:
        sections.append(section.elements)
    return sections

get_sections(result)

###  階層構造を生成

In [ ]:
# 入力データ
input_data = [
    ['/paragraphs/1', '/sections/1', '/sections/2', '/sections/5'],
    ['/paragraphs/2', '/paragraphs/3'],
    ['/paragraphs/4', '/sections/3', '/sections/4'],
    ['/paragraphs/5', '/paragraphs/6', '/tables/0'],
    ['/paragraphs/15', '/figures/0'],
    ['/paragraphs/37', '/paragraphs/38', '/paragraphs/39', '/paragraphs/40', '/paragraphs/41', '/paragraphs/42', '/paragraphs/43', '/paragraphs/44']
]

def explore_sections(input_data, indices, depth=0):
    indent = ' ' * depth  # 階層に応じたインデント
    for idx in indices:
        if idx < len(input_data):
            for path in input_data[idx]:
                print(indent + f"{idx}: {path}")
                if 'sections' in path:
                    number = int(path.split('/')[-1])
                    # 再帰的にさらにそのセクションを探索
                    explore_sections(input_data, [number], depth + 2)

def generate_hierarchy(input_data):
    initial_indices = [0]
    # 最初のリストの全要素を表示するために初期インデックスを0に設定し、そこから探索開始
    explore_sections(input_data, initial_indices)

# 階層構造を生成
generate_hierarchy(input_data)

## 画像
ドキュメント内の図形 (グラフ、イメージ) は、以下のように図の `caption`(存在する場合)、`boundingRegions` ドキュメント ページ上の図形の空間位置座標(pt)、`pageNumber` ページ番号、`elements` 図に関連する、または図を説明するドキュメント内のテキスト要素または段落の識別子などを取得できます。

In [ ]:
if result.figures:
    for idx, figures in enumerate(result.figures):
        print(f"--------Analysis of Figures #{idx + 1}--------")

        if figures.caption:
            title = figures.caption.get("content")
            if title:
                print(f"Caption: {title}")

            elements = figures.caption.get("elements")
            if elements:
                print("...caption elements involved:")
                for item in elements:
                  print(f"......Item #{item}")

            captionBR = []
            caption_boundingRegions = figures.caption.get("boundingRegions")
            if caption_boundingRegions:
                print("...caption bounding regions involved:")
                for item in caption_boundingRegions:
                    #print(f"...Item #{item}")
                    print(f"......Item pageNumber: {item.get('pageNumber')}")
                    print(f"......Item polygon: {item.get('polygon')}")
                    captionBR = item.get('polygon')

        if figures.elements:
            print("Elements involved:")
            for item in figures.elements:
                print(f"...Item #{item}")

        boundingRegions = figures.get("boundingRegions")
        if boundingRegions:
            print("Bounding regions involved:")
            for item in boundingRegions:
                #print(f"...Item #{item}")
                if captionBR != item.get('polygon'):
                    print(f"......Item pageNumber: {item.get('pageNumber')}")
                    print(f"......Item polygon: {item.get('polygon')}")


### 図の切り出しと保存
ちょうど [Microsoft techcommunity](https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/build-intelligent-rag-for-multimodality-and-complex-document/ba-p/4118184) で紹介されていたコードがすぐ使えるので引用します。[PyMuPDF](https://pymupdf.readthedocs.io/ja/latest/) ライブラリを利用して直接 PDF から画像として切り出しています。


In [ ]:
from PIL import Image
import fitz  # PyMuPDF
import mimetypes

import base64
from mimetypes import guess_type

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

def crop_image_from_image(image_path, page_number, bounding_box):
    """
    Crops an image based on a bounding box.

    :param image_path: Path to the image file.
    :param page_number: The page number of the image to crop (for TIFF format).
    :param bounding_box: A tuple of (left, upper, right, lower) coordinates for the bounding box.
    :return: A cropped image.
    :rtype: PIL.Image.Image
    """
    with Image.open(image_path) as img:
        if img.format == "TIFF":
            # Open the TIFF image
            img.seek(page_number)
            img = img.copy()
            
        # The bounding box is expected to be in the format (left, upper, right, lower).
        cropped_image = img.crop(bounding_box)
        return cropped_image

def crop_image_from_pdf_page(pdf_path, page_number, bounding_box):
    """
    Crops a region from a given page in a PDF and returns it as an image.

    :param pdf_path: Path to the PDF file.
    :param page_number: The page number to crop from (0-indexed).
    :param bounding_box: A tuple of (x0, y0, x1, y1) coordinates for the bounding box.
    :return: A PIL Image of the cropped area.
    """
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_number)
    
    # Cropping the page. The rect requires the coordinates in the format (x0, y0, x1, y1).
    # The coordinates are in points (1/72 inch).
    bbx = [x * 72 for x in bounding_box]
    rect = fitz.Rect(bbx)
    pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72), clip=rect)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    
    doc.close()

    return img

def crop_image_from_file(file_path, page_number, bounding_box):
    """
    Crop an image from a file.

    Args:
        file_path (str): The path to the file.
        page_number (int): The page number (for PDF and TIFF files, 0-indexed).
        bounding_box (tuple): The bounding box coordinates in the format (x0, y0, x1, y1).

    Returns:
        A PIL Image of the cropped area.
    """
    mime_type = mimetypes.guess_type(file_path)[0]
    
    if mime_type == "application/pdf":
        return crop_image_from_pdf_page(file_path, page_number, bounding_box)
    else:
        return crop_image_from_image(file_path, page_number, bounding_box)

In [ ]:
polygon = [1.0301, 7.1098, 4.1763, 7.1074, 4.1781, 9.0873, 1.0324, 9.0891]
bounding_box = (polygon[0], polygon[1], polygon[4], polygon[5])
image = crop_image_from_file("layout-pageobject.pdf", 0, bounding_box)
image.show()

In [ ]:
image.save("figure_1.png")